# Вариант 10

[[23126, u'en', u'Compass - powerful SASS library that makes your life easier'], [21617, u'en', u'Preparing for the AP* Computer Science A Exam \u2014 Part 2'], [16627, u'es', u'Aprende Excel: Nivel Intermedio by Alfonso Rinsche'], [11556, u'es', u'Aprendizaje Colaborativo by UNID Universidad Interamericana para el Desarrollo'], [16704, u'ru', u'\u041f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u0435 \u043d\u0430 Lazarus'], [13702, u'ru', u'\u041c\u0430\u0442\u0435\u043c\u0430\u0442\u0438\u0447\u0435\u0441\u043a\u0430\u044f \u044d\u043a\u043e\u043d\u043e\u043c\u0438\u043a\u0430']]

In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [16]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
conf.set("spark.app.name", "tatiana.inkhireeva 2") 

spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [69]:
df = spark.read.json('/labs/slaba02/DO_record_per_line.json')

In [72]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, ArrayType, FloatType

import pyspark.sql.functions as f

In [5]:
schema = StructType(fields=[
    StructField("id", LongType()),
    StructField("lang", StringType()),
    StructField("name", StringType()),
])

In [18]:
given_courses = spark.createDataFrame(
                schema=schema,
                data=[[23126, u'en', u'Compass - powerful SASS library that makes your life easier'], 
                 [21617, u'en', u'Preparing for the AP* Computer Science A Exam \u2014 Part 2'], 
                 [16627, u'es', u'Aprende Excel: Nivel Intermedio by Alfonso Rinsche'], 
                 [11556, u'es', u'Aprendizaje Colaborativo by UNID Universidad Interamericana para el Desarrollo'], 
                 [16704, u'ru', u'\u041f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u0435 \u043d\u0430 Lazarus'], 
                 [13702, u'ru', u'\u041c\u0430\u0442\u0435\u043c\u0430\u0442\u0438\u0447\u0435\u0441\u043a\u0430\u044f \u044d\u043a\u043e\u043d\u043e\u043c\u0438\u043a\u0430']]
)

In [19]:
given_courses = given_courses.persist()

Calculate TF-IDF

In [8]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, Normalizer

In [70]:
tokenizer = Tokenizer(inputCol="desc", outputCol="words")
df = tokenizer.transform(df)

hashingTF = HashingTF(inputCol="words", outputCol="tf", numFeatures=10000)
df = hashingTF.transform(df)

idf = IDF(inputCol="tf", outputCol="tfidf")
idfModel = idf.fit(df)
df = idfModel.transform(df)

normalizer = Normalizer(inputCol='tfidf', outputCol='tfidf_norm')
df = normalizer.transform(df)

Calculate cosine distance

In [73]:
dotProductUdf = f.udf(lambda v1, v2: float(v1.dot(v2)), FloatType())

23126

In [22]:
df = df.crossJoin(df.filter(f.col('id') == '23126').select(f.col('tfidf_norm').alias('c1')))

In [26]:
result = df.withColumn('cosine_sim', dotProductUdf('tfidf_norm', 'c1')).drop('tfidf', 'tfidf_norm', 'tf')\
.orderBy(f.col('cosine_sim').desc()).limit(15).toPandas()

/usr/hdp/current/spark2-client/python/pyspark/sql/dataframe.py:2111: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


In [28]:
output = dict()

In [32]:
output[result['id'][0].tolist()] = result['id'][1:11].tolist()

21617

In [31]:
result['id'][0].tolist()

23126

In [33]:
df = df.crossJoin(df.filter(f.col('id') == '21617').select(f.col('tfidf_norm').alias('c2')))

In [34]:
result = df.withColumn('cosine_sim', dotProductUdf('tfidf_norm', 'c2')).drop('tfidf', 'tfidf_norm', 'tf')\
.orderBy(f.col('cosine_sim').desc()).limit(15).toPandas()

/usr/hdp/current/spark2-client/python/pyspark/sql/dataframe.py:2111: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


In [36]:
output[result['id'][0].tolist()] = result['id'][1:11].tolist()

16627

In [ ]:
df = df.crossJoin(df.filter(f.col('id') == '16627').select(f.col('tfidf_norm').alias('c3')))

In [38]:
result = df.withColumn('cosine_sim', dotProductUdf('tfidf_norm', 'c3')).drop('tfidf', 'tfidf_norm', 'tf')\
.orderBy(f.col('cosine_sim').desc()).limit(15).toPandas()

/usr/hdp/current/spark2-client/python/pyspark/sql/dataframe.py:2111: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


In [41]:
output[result['id'][0].tolist()] = result['id'][1:11].tolist()

11556

In [50]:
df = df.crossJoin(df.filter(f.col('id') == '11556').select(f.col('tfidf_norm').alias('c4')))

In [51]:
result = df.withColumn('cosine_sim', dotProductUdf('tfidf_norm', 'c4')).drop('tfidf', 'tfidf_norm', 'tf')\
.orderBy(f.col('cosine_sim').desc()).limit(15).toPandas()

/usr/hdp/current/spark2-client/python/pyspark/sql/dataframe.py:2111: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


In [52]:
output[result['id'][0].tolist()] = result['id'][1:11].tolist()

16704

In [53]:
df = df.crossJoin(df.filter(f.col('id') == '16704').select(f.col('tfidf_norm').alias('c5')))

In [54]:
result = df.withColumn('cosine_sim', dotProductUdf('tfidf_norm', 'c5')).drop('tfidf', 'tfidf_norm', 'tf')\
.orderBy(f.col('cosine_sim').desc()).limit(15).toPandas()

/usr/hdp/current/spark2-client/python/pyspark/sql/dataframe.py:2111: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


In [55]:
output[result['id'][0].tolist()] = result['id'][1:11].tolist()

In [56]:
output

{23126: [13782, 13665, 24419, 20638, 2724, 25782, 2633, 2723, 13348, 15909],
 21617: [21609, 21608, 21616, 21492, 21624, 21623, 21630, 21628, 21508, 21857],
 16627: [11431, 12247, 13021, 25010, 11575, 5687, 9598, 5372, 12863, 16769],
 11556: [16488, 13461, 468, 10447, 387, 22710, 9289, 5936, 23357, 7833],
 16704: [3864, 23407, 1365, 20645, 20105, 1426, 8217, 1236, 1164, 23864]}

13702

In [74]:
df = df.crossJoin(df.filter(f.col('id') == '13702').select(f.col('tfidf_norm').alias('c6')))

In [75]:
result = df.withColumn('cosine_sim', dotProductUdf('tfidf_norm', 'c6')).drop('tfidf', 'tfidf_norm', 'tf')\
.orderBy(f.col('cosine_sim').desc()).limit(15).toPandas()

/usr/hdp/current/spark2-client/python/pyspark/sql/dataframe.py:2111: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


In [88]:
output[result['id'][1].tolist()] = [result['id'][0].tolist()] + result['id'][2:11].tolist()

In [86]:
result

,cat,desc,id,lang,name,provider,words,c6,cosine_sim
0,6/economics_finance|15/mathematics_statistics_...,Математическая экономика – это набор моделей в...,864,ru,Математическая экономика,Intuit,"[математическая, экономика, –, это, набор, мод...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.000000
1,6/economics_finance|15/mathematics_statistics_...,Математическая экономика – это набор моделей в...,13702,ru,Математическая экономика,Intuit,"[математическая, экономика, –, это, набор, мод...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.000000
2,13/physics|15/mathematics_statistics_and_data_...,В курсе дается введение в основные математичес...,1216,ru,Введение в математические модели механики спло...,Intuit,"[в, курсе, дается, введение, в, основные, мате...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.225345
3,5/computer_science,Интерактивный курс обучает основам программиро...,7173,ru,"Python, структуры данных для начинающих. (осно...",Acodemy,"[интерактивный, курс, обучает, основам, програ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.223182
4,6/economics_finance,Потребление сегодня – не просто средство обесп...,8313,ru,Российские потребители: как мы потребляем?,Universarium,"[потребление, сегодня, –, не, просто, средство...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.221726
5,3/business_management|6/economics_finance,В курсе рассмотрены классические понятия вопро...,1052,ru,Информационные технологии в экономике,Intuit,"[в, курсе, рассмотрены, классические, понятия,...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.219946
6,16/languages,Программа «Английский язык для путешествий» по...,17017,ru,Английский язык для путешествий,Lingvister,"[программа, «английский, язык, для, путешестви...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.212747
7,14/social_sciences,Прочувствуйте настоящую итальянскую дольче ви...,19613,ru,"Римские каникулы - с пользой, интересно и не з...",Udemy,"[, прочувствуйте, настоящую, итальянскую, доль...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.212036
8,,"Нанометр – это тот же метр, только в миллиард ...",21017,ru,Введение в нанотехнологии,Universarium,"[нанометр, –, это, тот, же, метр,, только, в, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.210413
9,16/languages,Программа «Английский язык для менеджеров по р...,17015,ru,Английский язык для менеджеров по работе с кли...,Lingvister,"[программа, «английский, язык, для, менеджеров...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.206292


In [89]:
output

{23126: [13782, 13665, 24419, 20638, 2724, 25782, 2633, 2723, 13348, 15909],
 21617: [21609, 21608, 21616, 21492, 21624, 21623, 21630, 21628, 21508, 21857],
 16627: [11431, 12247, 13021, 25010, 11575, 5687, 9598, 5372, 12863, 16769],
 11556: [16488, 13461, 468, 10447, 387, 22710, 9289, 5936, 23357, 7833],
 16704: [3864, 23407, 1365, 20645, 20105, 1426, 8217, 1236, 1164, 23864],
 13702: [864, 1216, 7173, 8313, 1052, 17017, 19613, 21017, 17015, 8082]}

In [90]:
import json
with open('lab02.json', 'w') as f:
    json.dump(output, f)

In [92]:
spark.stop()